In [1]:
import sys
sys.path.insert(0, "HelperFunctions")
from sklearn.model_selection import train_test_split
from UNetModels import UNet
from PlotLearningModels import plotLoss
from LossFunctions import dice_coef_loss, dice_coef
from GenerateTrainingDataSingleChannel import AugmentImages, LoadTrainingData, getTrainingTiles
import numpy as np

Using TensorFlow backend.


In [2]:
ImageDirectory = '/data/u934/service_imagerie/v_kapoor/from_Ozge_for_curvature/multiple_cells/images/'
MaskDirectory = '/data/u934/service_imagerie/v_kapoor/from_Ozge_for_curvature/multiple_cells/masks/'

Input_Channels = 1
Output_Channels = 1
batchsize = 8
epochs = 40
reload = False
##size of image patches for training 
## note: must be divisible by 32 for UNet 

PATCH_HEIGHT, PATCH_WIDTH = 1024,1024
Num_Samples = 4


Augment training data, meaning create more images from the given input images (optional block)

In [3]:
AugmentImages(ImageDirectory, MaskDirectory)

Zero padding all images to the max size found:  2048 2048


HelperFunctions/Normalize.py:48: UserWarning: Converting data type from 'float64' to ImageJ-compatible 'float32'.
  warnings.warn("Converting data type from '%s' to ImageJ-compatible '%s'." % (t, np.dtype(t_new)))
HelperFunctions/Normalize.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  x = x[slices]
HelperFunctions/Normalize.py:48: UserWarning: Converting data type from 'int64' to ImageJ-compatible 'int16'.
  warnings.warn("Converting data type from '%s' to ImageJ-compatible '%s'." % (t, np.dtype(t_new)))


Load the training data

In [4]:

rankfourX, rankfourY = LoadTrainingData(ImageDirectory, MaskDirectory)


X, Y = getTrainingTiles(rankfourX, rankfourY,PATCH_HEIGHT,PATCH_WIDTH, Num_Samples)

(40, 2048, 2048, 1)


In [5]:

x_validation, x_train, y_validation, y_train = train_test_split(X, Y, test_size=.8)
print('Training on input tensor: {}'.format(x_train.shape))
print('Validating on input tensor: {}'.format(x_validation.shape))


Training on input tensor: (128, 1024, 1024, 1)
Validating on input tensor: (32, 1024, 1024, 1)


In [ ]:
model_unet = UNet(n_input_channels=Input_Channels, n_output_channels=Output_Channels)
model_unet.compile(optimizer='adam', loss=dice_coef_loss, metrics=[dice_coef])
history_unet = model_unet.fit(x_train, y_train, batch_size=batchsize, epochs=epochs, 
                    verbose=1, shuffle=True, 
                    validation_data=(x_validation, y_validation),
                    callbacks=[plotLoss])

Train on 128 samples, validate on 32 samples
Epoch 1/40
